## _*Using Qiskit Aqua for the vertex cover problems*_

A vertex cover of a graph is a set of vertices such that each edge of the graph is incident to at least one vertex of the set. The goal of NPC problem is to minimize the size of the vertex cover. 


We will go through three examples to show (1) how to run the optimization in the non-programming way, (2) how to run the optimization in the programming way, (3) how to run the optimization with the VQE.
We will omit the details for the support of CPLEX, which are explained in other notebooks such as maxcut.


### The problem and the brute-force method

the problem is as follows. the graph is in the adjacent matrix form.

In [1]:
import numpy as np
from qiskit import Aer
from qiskit_aqua import run_algorithm
from qiskit_aqua.input import EnergyInput
from qiskit_aqua.translators.ising import vertexcover
from qiskit_aqua.algorithms import ExactEigensolver

np.random.seed(100)
num_nodes = 3
w = vertexcover.random_graph(num_nodes, edge_prob=0.8, weight_range=10)
print(w)
qubit_op, offset = vertexcover.get_vertexcover_qubitops(w)
algo_input = EnergyInput(qubit_op)

[[0. 4. 5.]
 [4. 0. 3.]
 [5. 3. 0.]]


the brute-force method is as follows. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a vertex is either 0 (meaning the vertex is not in the cover) or 1 (meaning the vertex is in the cover). We print the binary assignment that satisfies the definition of the vertex cover and corresponds to the minimial size. 

In [2]:
def brute_force():
    # brute-force way
    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

    L = num_nodes
    max = 2**L
    minimal_v = np.inf
    for i in range(max):
        cur = bitfield(i, L)

        cur_v = vertexcover.check_full_edge_coverage(np.array(cur), w)
        if cur_v:
            nonzerocount = np.count_nonzero(cur)
            if nonzerocount < minimal_v:
                minimal_v = nonzerocount

    return minimal_v

size = brute_force()
print('size of the vertex cover', size)

size of the vertex cover 2


### Part I: run the optimization in the non-programming way

In [3]:
params = {
    'problem': {'name': 'ising'},
    'algorithm': {'name': 'ExactEigensolver'}
}
result = run_algorithm(params, algo_input)

x = vertexcover.sample_most_likely(len(w), result['eigvecs'][0])
sol = vertexcover.get_graph_solution(x)
np.testing.assert_array_equal(sol, [0, 1, 1])
print('size of the vertex cover', np.count_nonzero(sol))

size of the vertex cover 2


### Part II: run the optimization in the programming way

In [4]:
algo = ExactEigensolver(algo_input.qubit_op, k=1, aux_operators=[])
result = algo.run()
x = vertexcover.sample_most_likely(len(w), result['eigvecs'][0])
sol = vertexcover.get_graph_solution(x)
np.testing.assert_array_equal(sol, [0, 1, 1])
print('size of the vertex cover', np.count_nonzero(sol))

size of the vertex cover 2


### Part III: run the optimization with VQE

In [5]:
algorithm_cfg = {
    'name': 'VQE',
    'operator_mode': 'paulis'
}

optimizer_cfg = {
    'name': 'SPSA',
    'max_trials': 200
}

var_form_cfg = {
    'name': 'RYRZ',
    'depth': 3,
}

params = {
    'problem': {'name': 'ising', 'random_seed': 100},
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg,
    'variational_form': var_form_cfg
}
backend = Aer.get_backend('qasm_simulator')
result = run_algorithm(params, algo_input, backend=backend)
x = vertexcover.sample_most_likely(len(w), result['eigvecs'][0])
sol = vertexcover.get_graph_solution(x)
print('size of the vertex cover', np.count_nonzero(sol))

size of the vertex cover 2
